In [35]:
TECH_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Tech'
CULTURE_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Culture'
Finance_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Finance'
Medical_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Medical'
politics_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Politics'
Religion_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Religion'
Sports_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Sports'

DOC_DIRECTORY = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant - Copy\data\processed\all_split.pkl'
# Sample_DIRECTORY = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\processed\sys_sample.pkl'


# Specify the directory where you want to save the vector database
PERSIST_DIRECTORY = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant - Copy\data\processed\vector_db'

In [2]:
import os
import pickle
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from tqdm import tqdm  # Import tqdm for the progress bar


c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# Functions

In [36]:

def read_text_files_in_folder(folders_paths: list):
    """
    Reads text files from multiple folders and returns a list of Document objects.

    Args:
        folders_paths (list): A list of folder paths containing text files.

    Returns:
        all_docs (list): A list of Document objects, where each document contains
                         the text content of a file and its metadata with the file path.

    The function iterates over each folder in the provided list of folder paths. It reads
    all `.txt` files within each folder, creates a Document object for each file with the
    text content and the file's metadata, and appends it to the list of documents.
    A progress bar is displayed for each folder to indicate the processing status.
    """
    all_docs = []
    for folder_path in folders_paths:
        # Use tqdm to add a progress bar around the list of files
        for file_name in tqdm(os.listdir(folder_path), desc=f"Processing {os.path.basename(folder_path)}"):
            if file_name.endswith('.txt'):
                file_path = os.path.join(folder_path, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read()
                    
                    source = file_path.split('\\')
                    source = f'{source[-2]}\\{source[-1]}' 
                    # Create a Document object for each text
                    doc = Document(page_content=text, metadata={"source": source})
                    all_docs.append(doc)
    return all_docs

# ----------------------------------------------------------------


def create_chroma_batches(documents, embedding_model, persist_directory, batch_size=100):
    """
    Processes documents in batches and creates a Chroma vector store with embeddings.

    Args:
        documents (list): A list of Document objects to be processed.
        embedding_model (SentenceTransformer): The embedding model used to generate embeddings for the documents.
        persist_directory (str): The directory where the Chroma vector store will be persisted.
        batch_size (int, optional): The maximum number of documents to process in each batch. Default is 100.

    Returns:
        chroma_instance (Chroma): The Chroma vector store instance containing the embedded documents.

    The function splits the documents into smaller batches and processes each batch to create
    a Chroma vector store with embeddings. A progress bar is used to show the progress of batch processing.
    If a Chroma instance does not exist, it is created with the first batch. Subsequent batches are added
    to the existing Chroma instance.
    """
    chroma_instance = None
    # client = Chroma.PersistentClient(path='db')
    # collection = client.get_collection(name="langchain", embedding_function=openai_ef)

    # Initialize the progress bar
    num_batches = len(documents) // batch_size + int(len(documents) % batch_size != 0)
    with tqdm(total=num_batches, desc="Processing batches", unit="batch") as pbar:
        # Process documents in batches
        for i in range(0, len(documents), batch_size):
            batch = documents[i:i + batch_size]
            
            if chroma_instance is None:
                # Create a new Chroma instance with the first batch
                chroma_instance = Chroma.from_documents(
                    documents=batch,
                    embedding=embedding_model,
                    persist_directory=persist_directory
                )
            else:
                # Add the batch to the existing Chroma instance
                chroma_instance.add_documents(documents=batch)
            
            # Update the progress bar
            pbar.update(1)

    return chroma_instance





all-distilroberta-v1'

- Average Performance: 59.76
- Speed: 4000 (fast)
- Model Size: 290 MB
- Strengths: High average performance with a good balance between speed and model size.

In [9]:
# Initialize the model 
model = SentenceTransformer('all-distilroberta-v1')

class embedding:
    def __init__(self):
        self.model = model
    def embed_documents(self , docs):
        embeddings = self.model.encode(docs)
        return embeddings.tolist()
    
    def embed_query(self , query):
        return self.model.encode(query).tolist()

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--sentence-transformers--all-distilroberta-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


------

# Load the data

### Because the data already split into small pieces and each base in different file so we don't need to chunk them

In [37]:
folders_paths = [TECH_FOLDER_PATH ,
                 CULTURE_FOLDER_PATH , 
                 Finance_FOLDER_PATH , 
                 Medical_FOLDER_PATH,
                 politics_FOLDER_PATH ,
                 Sports_FOLDER_PATH, 
                 Religion_FOLDER_PATH 
                 ]
folders_paths[0].split('\\')[-1]

'Tech'

In [38]:
all_split =  read_text_files_in_folder(folders_paths)

Processing Religion: 100%|██████████| 6500/6500 [00:08<00:00, 737.83it/s] 


In [23]:
# all_split[10000:]

### store all doc

In [39]:
import pickle

# Save the all_split list to a file
with open(DOC_DIRECTORY, 'wb') as f:
    pickle.dump(all_split, f)

In [40]:


# Load the all_split list from the file
with open(DOC_DIRECTORY, 'rb') as f:
    all_split = pickle.load(f)

In [41]:
print(f"Total documents: {len(all_split)}")

Total documents: 45500


In [7]:
6499 * 7

45493

# Embedding

In [42]:
embed_model = embedding()

len(embed_model.embed_query('مرحبا بك'))

768

# Vector DataBase

### Chroma maximum batch size 41666 so we will take 41666 record only 

In [12]:
# all_split = sys_sampling

In [28]:
len(all_split)

45500

In [43]:
# Process your documents in smaller batches with a progress bar
vector_data = create_chroma_batches(
    documents = all_split,
    embedding_model = embed_model,
    persist_directory = PERSIST_DIRECTORY,
    batch_size = 5  # Set a batch size within the limit
)


Processing batches: 100%|██████████| 9100/9100 [22:26:48<00:00,  8.88s/batch]      


In [44]:
reteiever = vector_data.as_retriever(search_type = 'similarity' , search_kwargs= {'k' : 2})
all_split[1].page_content

'شهدت منصة (My-HD) التلفزيونية نمواً ثابتاً في السوق الإقليمية خلال الفترة القصيرة الماضية . وقال كليف نيلسون، الرئيس التنفيذي لشركة ماي إتش دي، إن عدد المشتركين من جميع دول المنطقة قد تجاوز 000 .100 مشترك . وخلال السنة الماضية أضافت ماي إتش دي 68 قناة جديدة لمجموعة القنوات التي تقوم ببثها، وهو ما يعكس التحول المتسارع من البث التلفزيوني التقليدي (SD) إلى البث عالي الوضوح (HD).'

In [47]:
# Later, you can load the database like this:
loaded_vector_data = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function = embed_model
)
reteiever = loaded_vector_data.as_retriever(search_type = 'similarity' , search_kwargs= {'k' : 3})
all_split[-1]

Document(metadata={'source': 'Religion\\6499.txt'}, page_content='وجود قدر من الغيرة قد يحرك الماء الراكد في العلاقة بين الزوجين، وعلى مقدار هذه الغيرة يتحدد شكل العلاقة، واختفاؤها قد يعني أن الحب والإحساس بالمسؤولية معدومان، وتعتبر الغيرة من توابل الحب إذا كانت بالمقدار المعتدل، وهي مطلوبة شرعاً وعرفاً، إلا أن منها ما يقتل صاحبه ويدمر حياته أو يتحول إلى قنبلة موقوتة قد تنفجر في أية لحظة فتدمر الأسرة كلها .والفارق بين الغيرة الملتهبة والشك خيط رفيع، يفشل بعض الأزواج والزوجات في التفرقة بينهما، ويجب ألا تفسد تلك الغيرة حياة صاحبها ومن حوله، وفي بعض الأحيان تكون نتيجتها نهاية مأساوية، ويبدأ الندم في وقت لا يفيد فيه الندم .الدكتور محمد نبيل غنايم، أستاذ الشريعة الإسلامية بكلية دار العلوم في جامعة القاهرة، يؤكد أن الغيرة طبيعة جبل عليها الإنسان السوي الذي كرمه ربه وفضله، وقد أعلى شأنها وقدرها الإسلام، وتظهر هذه الصفة والغريزة الإنسانية أكثر ما تظهر بين الزوجين، لأنها يشترك فيها الرجال والنساء، بل قد تكون أكثر وأشد لدى المرأة .والغيرة في موضعها والاعتدال فيها بالنسبة للرجال والنساء من جملة 

In [48]:
q = all_split[-1].page_content
reteiever.invoke(q)


[Document(metadata={'source': 'Religion\\6499.txt'}, page_content='وجود قدر من الغيرة قد يحرك الماء الراكد في العلاقة بين الزوجين، وعلى مقدار هذه الغيرة يتحدد شكل العلاقة، واختفاؤها قد يعني أن الحب والإحساس بالمسؤولية معدومان، وتعتبر الغيرة من توابل الحب إذا كانت بالمقدار المعتدل، وهي مطلوبة شرعاً وعرفاً، إلا أن منها ما يقتل صاحبه ويدمر حياته أو يتحول إلى قنبلة موقوتة قد تنفجر في أية لحظة فتدمر الأسرة كلها .والفارق بين الغيرة الملتهبة والشك خيط رفيع، يفشل بعض الأزواج والزوجات في التفرقة بينهما، ويجب ألا تفسد تلك الغيرة حياة صاحبها ومن حوله، وفي بعض الأحيان تكون نتيجتها نهاية مأساوية، ويبدأ الندم في وقت لا يفيد فيه الندم .الدكتور محمد نبيل غنايم، أستاذ الشريعة الإسلامية بكلية دار العلوم في جامعة القاهرة، يؤكد أن الغيرة طبيعة جبل عليها الإنسان السوي الذي كرمه ربه وفضله، وقد أعلى شأنها وقدرها الإسلام، وتظهر هذه الصفة والغريزة الإنسانية أكثر ما تظهر بين الزوجين، لأنها يشترك فيها الرجال والنساء، بل قد تكون أكثر وأشد لدى المرأة .والغيرة في موضعها والاعتدال فيها بالنسبة للرجال والنساء من جملة